In [ ]:
#!pip install pypdf
#!pip install python-dotenv

In [1]:
!nvidia-smi

Sat Dec 23 00:08:19 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    On  | 00000000:00:1E.0 Off |                    0 |
|  0%   17C    P8              15W / 300W |      2MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip -q install git+https://github.com/huggingface/transformers

In [3]:
!pip install -q datasets loralib sentencepiece

In [4]:
!pip install -q einops accelerate langchain bitsandbytes

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.1 which is incompatible.
sphinx 7.2.6 requires docutils<0.21,>=0.18.1, but you have docutils 0.16 which is incompatible.


In [5]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.5 MB/s eta 0:00:00ta 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 11.6 MB/s eta 0:00:0000:0100:01
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=a7521a2e754f9bf3cf1ca5b15e0988812d055754bb1b29f75db334bab0c67f34
  Stored in directory: /home/ec2-user/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [6]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 14.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 6.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 1.0 MB/s eta 0:00:00eta 0:00:01


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.7 MB/s eta 0:00:00:00:01


In [7]:
#pip install -U flash-attn --no-build-isolation

In [12]:

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM

In [13]:
#documents = SimpleDirectoryReader("example_data").load_data()

In [14]:
def get_meta(file_path):
    return {"file_name": file_path.split("/")[-1]}

reader = SimpleDirectoryReader(input_dir="/home/ec2-user/SageMaker/VegRD_APD_Fruit_Phenotyping/scripts",recursive=True,
                              required_exts=[".py"],file_metadata=get_meta)

documents = reader.load_data()

In [15]:
len(documents)

35

In [16]:
documents[0]

Document(id_='8b22e4ba-bcbd-4929-940f-666d167efdbb', embedding=None, metadata={'file_path': '/home/ec2-user/SageMaker/VegRD_APD_Fruit_Phenotyping/scripts/Hot_Peppers.py', 'file_name': 'Hot_Peppers.py', 'file_type': 'text/x-python', 'file_size': 20914, 'creation_date': '2023-12-20', 'last_modified_date': '2023-12-20', 'last_accessed_date': '2023-12-23'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='d2623851cefba477369443c24ced9fac58318b3b800ab4757dfaeaacbab9a2f0', text='import os\nimport cv2\nimport collections\nimport numpy as np\nimport pandas as pd\nimport boto3\nimport warnings\nfrom tqdm import tqdm\nfrom statistics import median\nfrom skimage.morphology import medial_axis\nfrom scipy.interpolate import splprep, splev\nfrom scipy.ndimage import di

In [17]:

#!huggingface-cli login


In [18]:
from llama_index.prompts.prompts import SimpleInputPrompt
system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = "<|USER|>{query_str}<|ASSISTANT|>"

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "auto" # the device to load the model onto


llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=1000,
    generate_kwargs={"temperature": 0.1, "do_sample": True},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map=device,
    tokenizer_kwargs={"max_length": 4096},
    model_kwargs={"torch_dtype": torch.float32},

)
     
#llm = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", torch_dtype=torch.float16)
#tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

#prompt = "My favourite condiment is"

#model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
#model.to(device)

#generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
#tokenizer.batch_decode(generated_ids)[0]

NameError: name 'HuggingFaceLLM' is not defined

In [ ]:
llm.model.parameters()

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
embed_model =HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2",model_kwargs = {'device': 'cpu'})

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
dir(embed_model)

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__validators__',
 '__weakref__',
 '_abc_impl',
 '_calculate_keys',
 '_copy_and_set_values',
 '_decompose

In [10]:
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
embed_model = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-base-en",model_kwargs = {'device': 'cpu'})

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en
Load pretrained SentenceTransformer: BAAI/bge-base-en


In [11]:
#!python -m pip install -U angle-emb

In [12]:
## SOTA Embeddings from huggingface leaderboard

from typing import Any, List
from llama_index.embeddings.base import BaseEmbedding
from llama_index.bridge.pydantic import PrivateAttr

## Embedding model specific current SOTA in huggung face leaderboard
from angle_emb import AnglE, Prompts


class UAEmbeddings(BaseEmbedding):
    
    _model: Any= PrivateAttr()
   
    def __init__(
        self,
        instructor_model_name: str = 'WhereIsAI/UAE-Large-V1',
        **kwargs: Any,
    ) -> None:
        
        super().__init__(**kwargs)  
        self._model=AnglE.from_pretrained(instructor_model_name, pooling_strategy='cls',)
        #self._model.set_prompt(prompt=None)

        
    def _aget_query_embedding(self, query: str) -> List[float]:
        # Implementation of the abstract method
        embeddings = self._model.encode([ query])
        return embeddings[0]
        
    def _get_query_embedding(self, query: str) -> List[float]:
        embeddings = self._model.encode([query])
        return embeddings[0]

    def _get_text_embedding(self, text: str) -> List[float]:
        embeddings = self._model.encode([text])
        return embeddings[0]

    def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
        embeddings = self._model.encode(
            [text for text in texts]
        )
        return embeddings

In [13]:
embed_model=UAEmbeddings()
embed_model._model.device
#embeddings = embed_model._get_text_embedding("It is raining cats and dogs here!")
#embeddings.shape

'cuda'

In [3]:
!pip install InstructorEmbedding


In [ ]:
from typing import Any, List
from InstructorEmbedding import INSTRUCTOR

from llama_index.bridge.pydantic import PrivateAttr
from llama_index.embeddings.base import BaseEmbedding


class InstructorEmbeddings(BaseEmbedding):
    _model: INSTRUCTOR = PrivateAttr()
    _instruction: str = PrivateAttr()

    def __init__(
        self,
        instructor_model_name: str = "hkunlp/instructor-large",
        instruction: str = "Represent code base to easily search from:",
        **kwargs: Any,
    ) -> None:
        self._model = INSTRUCTOR(instructor_model_name,device="cpu")
        self._instruction = instruction
        super().__init__(**kwargs)

    @classmethod
    def class_name(cls) -> str:
        return "instructor"

    async def _aget_query_embedding(self, query: str) -> List[float]:
        return self._get_query_embedding(query)

    async def _aget_text_embedding(self, text: str) -> List[float]:
        return self._get_text_embedding(text)

    def _get_query_embedding(self, query: str) -> List[float]:
        embeddings = self._model.encode([[self._instruction, query]])
        return embeddings[0]

    def _get_text_embedding(self, text: str) -> List[float]:
        embeddings = self._model.encode([[self._instruction, text]])
        return embeddings[0]

    def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
        embeddings = self._model.encode(
            [[self._instruction, text] for text in texts]
        )
        return embeddings

In [ ]:
embed_model=InstructorEmbeddings()
#embeddings = embed_model._get_text_embedding("It is raining cats and dogs here!")

In [ ]:
embeddings.shape

In [21]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [22]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
query_engine = index.as_query_engine()

In [26]:
response = query_engine.query("what is algorithm to calculate c-length of hot peppers?")
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The algorithm to calculate c-length of hot peppers is as follows:

1. Convert pixels to real units cm/mm if ppx and ppy are defined, otherwise find lengths of curves.
2. Process the segments of the hot pepper by finding the top, bottom, and widths of each segment.
3. Calculate the shape metrics of the hot pepper, including the c-length, using the processed segments.

The c-length is calculated by dividing the length of the hot pepper by the width of the mid-height region.


In [ ]:
response = query_engine.query("How to turn on/off visulaizations of phenotypes?")
print(response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [25]:
response = query_engine.query("How to turn on/off visulaizations of detections and phenotypes of hot peppers lsit all methods?")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You can turn on/off visualizations of detections and phenotypes of hot peppers by setting the vis parameter to True or False respectively. The following methods are available to turn on/off visualizations:

* phenotype\_measurement\_pepper(img, patch, fname, fruitIdx, vis=False, ppx=None, ppy=None)
* run\_inference(det, learn, img, vis=False)

You can call these methods with the vis parameter set to True or False to turn on/off visualizations.
